In [1]:
# So can get yadlt
import sys
sys.path.append('/usr2/mamille2/nn4nlp-scripts/mm_dbm/Deep-Learning-TensorFlow/')

import pandas as pd
from collections import defaultdict, Counter
import numpy as np
import tensorflow as tf
import inspect
import fasttext as ft
from nltk import word_tokenize
import nltk
import string
import pickle
from tqdm import tqdm_notebook as tqdm

import dbm
import config
import rbm

# Pretraining

In [2]:
# Vocab in words
with open('/usr2/mamille2/twitter/data/huang2016_data/train_text_vocab_10k.txt') as f:
    vocab = f.read().splitlines()
    
# Vocab embeddings
with open('/usr2/mamille2/twitter/data/huang2016_data/train_vocab_10k_emb.pkl', 'rb') as g:
    embs = pickle.load(g)

In [3]:
# Training text, preprocessed
with open('/usr2/mamille2/twitter/data/huang2016_data/text_train.txt') as f:
    toks_train = [l.split() for l in f.read().splitlines()]
    
print(len(toks_train))
# print(toks_train[0])

214999


In [10]:
# Modeled on image data

model = dbm.DBM(
    main_dir="/usr2/mamille2/twitter/data/huang2016_data/twitter_text_rbm", do_pretrain=True, layers=[1024, 1024],
    models_dir=config.models_dir, data_dir=config.data_dir, summary_dir=config.summary_dir,
    learning_rate=[0.001, 0.001], momentum=0.9, num_epochs=[1, 1], batch_size=[64, 64],
    stddev=0.1, verbose=1, gibbs_k=[1, 1], model_name="twitter_text_dbm",
    finetune_learning_rate=0.01, finetune_enc_act_func=[tf.nn.sigmoid, tf.nn.sigmoid],
    finetune_dec_act_func=[tf.nn.sigmoid, tf.nn.sigmoid], finetune_num_epochs=5, finetune_batch_size=128,
    finetune_opt='momentum', finetune_loss_func="mean_squared", finetune_dropout=0.5, noise=["gauss", "bin"],
)

Creating /usr2/mamille2/twitter/data/mm_dbm/model/stored_models/ directory to save/restore models
Creating /usr2/mamille2/twitter/data/mm_dbm/model/data/ directory to save model generated data
Creating /usr2/mamille2/twitter/data/mm_dbm/model/logs/ directory to save tensorboard logs
Creating /usr2/mamille2/twitter/data/mm_dbm/model/stored_models/rbm-1 directory to save/restore models
Creating /usr2/mamille2/twitter/data/mm_dbm/model/data/rbm-1 directory to save model generated data
Creating /usr2/mamille2/twitter/data/mm_dbm/model/logs/rbm-1 directory to save tensorboard logs
Creating /usr2/mamille2/twitter/data/mm_dbm/model/stored_models/rbm-2 directory to save/restore models
Creating /usr2/mamille2/twitter/data/mm_dbm/model/data/rbm-2 directory to save model generated data
Creating /usr2/mamille2/twitter/data/mm_dbm/model/logs/rbm-2 directory to save tensorboard logs


In [5]:
MAX_LEN = 80 # Max number of tweets around 80

# Construct UNK vector of average vector
all_embs = np.array(list(embs.values())[:])
unkvec = np.mean(all_embs, axis=0)

def tweet2vec(tweet_toks, embs, max_len):
    """ Return concatenated word embeddings from tweet tokens, padded to a max length """
    
    emb_list = [embs.get(w, unkvec) for w in tweet_toks[:max_len]]
    tweet_mat = np.pad(emb_list, ((0,max_len-len(emb_list)), (0,0)), 'constant', constant_values=0)
    
    return tweet_mat.flatten()

# Get data dynamically so not so much RAM usage
def tweets_embed(tweets_toks, embs, max_len):
    """ Return word embeddings from list of tweet tokens, padded to a max length """
    
    tweets_mat = [tweet2vec(tweet, embs, max_len) for tweet in tweets_toks]
        
    return np.array(tweets_mat)

In [11]:
# Pretraining phase 
# train1 = tweets_embed(toks_train[:500], embs, MAX_LEN)
# train2 = tweets_embed(toks_train[500:1000], embs, MAX_LEN)
train1 = tweets_embed(toks_train[:10000], embs, MAX_LEN)
print("Done building train 1")
train2 = tweets_embed(toks_train[10000:20000], embs, MAX_LEN)
print("Done building train 2")
# print(train1.shape)
# print(train2.shape)

Done building train 1
Done building train 2


In [12]:
model.pretrain(train1, train2)

Training layer 1...
Tensorboard logs dir for this run is /usr2/mamille2/twitter/data/mm_dbm/model/logs/rbm-1/run20
Reconstruction loss at step 0: 0.404077
Training layer 2...
Tensorboard logs dir for this run is /usr2/mamille2/twitter/data/mm_dbm/model/logs/rbm-2/run19
Reconstruction loss at step 0: 0.118569


(array([[ 0.56180596,  0.66380399,  0.31962618, ...,  0.54801655,
          0.43488625,  0.4274264 ],
        [ 0.58443767,  0.57850999,  0.38086259, ...,  0.61280924,
          0.48586884,  0.36361116],
        [ 0.65635574,  0.63681233,  0.32367343, ...,  0.4517298 ,
          0.50786972,  0.35507789],
        ..., 
        [ 0.67270261,  0.62787724,  0.4213872 , ...,  0.37202278,
          0.48151723,  0.4435603 ],
        [ 0.40214041,  0.53491944,  0.42307886, ...,  0.48610663,
          0.56079775,  0.37639761],
        [ 0.66140223,  0.50370604,  0.56105483, ...,  0.35186243,
          0.25721386,  0.3114686 ]], dtype=float32),
 array([[ 0.60677838,  0.57522899,  0.59163254, ...,  0.66177225,
          0.55448431,  0.4701899 ],
        [ 0.57875955,  0.64811701,  0.40937036, ...,  0.2916674 ,
          0.61565715,  0.38740188],
        [ 0.64729851,  0.83547252,  0.36572573, ...,  0.43609035,
          0.55515784,  0.28614867],
        ..., 
        [ 0.66834635,  0.70633572,  0

In [13]:
# Fit and save the txt-DBM parameters 
# I put save_dbm_text_params as a quick hack to save the parameters of this dbm as a numpy array

# takes args train_set, train_ref, validation_set, validation_ref -- unsupervised, so just use more of the dataset
# train1 = tweets_embed(toks_train[:500], embs, MAX_LEN)
# train2 = tweets_embed(toks_train[500:1000], embs, MAX_LEN)
# train3 = tweets_embed(toks_train[1000:1500], embs, MAX_LEN)
# train4 = tweets_embed(toks_train[1500:2000], embs, MAX_LEN)

# train_split = len(toks_train)//4
train_split = 10000

train1 = tweets_embed(toks_train[:train_split], embs, MAX_LEN)
print("Train 1 loaded")
train2 = tweets_embed(toks_train[train_split:2*train_split], embs, MAX_LEN)
print("Train 2 loaded")
train3 = tweets_embed(toks_train[2*train_split:3*train_split], embs, MAX_LEN)
print("Train 3 loaded")
train4 = tweets_embed(toks_train[3*train_split:4*train_split], embs, MAX_LEN)
print("Train 4 loaded")

Train 1 loaded
Train 2 loaded
Train 3 loaded
Train 4 loaded


In [14]:
model.fit(train1, train2, train3, train4)

Tensorboard logs dir for this run is /usr2/mamille2/twitter/data/mm_dbm/model/logs/run17
Reconstruction loss at step 0: 0.381881
Reconstruction loss at step 1: 0.381118
Reconstruction loss at step 2: 0.38041
Reconstruction loss at step 3: 0.379778
Reconstruction loss at step 4: 0.379198


In [17]:
# Fit and save the txt-DBM parameters (iterative try)

offset = 4 * train_split
increment = 10000

train1 = tweets_embed(toks_train[offset:offset+increment], embs, MAX_LEN)
print("Train 1 loaded")
train2 = tweets_embed(toks_train[offset+increment:offset+2*increment], embs, MAX_LEN)
print("Train 2 loaded")
train3 = tweets_embed(toks_train[offset+2*increment:offset+3*increment], embs, MAX_LEN)
print("Train 3 loaded")
train4 = tweets_embed(toks_train[offset+3*increment:offset+4*increment], embs, MAX_LEN)
print("Train 4 loaded")

Train 1 loaded
Train 2 loaded
Train 3 loaded
Train 4 loaded


In [19]:
model.fit(train1, train2, train3, train4, restore_previous_model=True)
print("\nDone with {} instances".format(offset + 4*increment))

FailedPreconditionError: Attempting to use uninitialized value enc-w-0
	 [[Node: enc-w-0/read = Identity[T=DT_FLOAT, _class=["loc:@enc-w-0"], _device="/job:localhost/replica:0/task:0/cpu:0"](enc-w-0)]]

Caused by op 'enc-w-0/read', defined at:
  File "/usr0/home/mamille2/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr0/home/mamille2/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.p

In [10]:
# Save out model params
outpath = '/usr2/mamille2/twitter/data/mm_dbm/model/text_dbms_layer_{}_{}.npy'

for i in range(len(model.rbms)):
    for p in ['W', 'bh_', 'bv_']: # not sure whether uses bh or bv
        np.save(outpath.format(i, p), model.rbms[0].get_model_parameters(graph=model.rbm_graphs[0])[p])

In [16]:
model.rbms[0].get_model_parameters(graph=model.rbm_graphs[0])['W'].shape

(5, 1024)

In [17]:
model.rbms[1].get_model_parameters(graph=model.rbm_graphs[1])['W'].shape

(1024, 1024)

In [18]:
model.rbms[0].get_model_parameters(graph=model.rbm_graphs[0])['bh_'].shape

(1024,)

In [19]:
model.rbms[0].get_model_parameters(graph=model.rbm_graphs[0])['bv_'].shape

(5,)

# Format tweet text as word embeddings

In [12]:
train_data = pd.read_pickle('/usr2/mamille2/twitter/data/huang2016_data/huang2016_train.pkl')
valid_data = pd.read_pickle('/usr2/mamille2/twitter/data/huang2016_data/huang2016_valid.pkl')
test_data = pd.read_pickle('/usr2/mamille2/twitter/data/huang2016_data/huang2016_test.pkl')
train_data.columns

Index(['tweet_id', 'created_at', 'in_reply_to_status_id', 'lang',
       'retweet_count', 'user_id', 'user_screen_name', 'user_name', 'text',
       'text_no_tags', 'tags'],
      dtype='object')

In [83]:
# Initialize word embeddings, make lookup table
wembed = ft.load_model('/usr2/mamille2/discourse_connectives/en_wiki_stanford_model_300.bin')
toks = train_data['text_no_tags'].map(lambda x: word_tokenize(x.lower())).tolist()

KeyboardInterrupt: 

In [18]:
custom_stops = ['amp', 'https', 'http', '...', '``', "''", "'s"]
stops = nltk.corpus.stopwords.words('english') + [c for c in string.punctuation] + custom_stops

# Stop patterns--reject if matches athg in them
stop_pats = ['…', 't.co']

print(stops)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [19]:
toks_agg = sorted([wd for d in toks for wd in d if not wd in stops and not any([s in wd for s in stop_pats])])
toks_ctr = Counter(toks_agg)
vocab_all = sorted(toks_ctr.keys())
len(vocab_all)

216704

In [20]:
vocab_10k = sorted([w for w,_ in toks_ctr.most_common(10000)])
len(vocab_10k)

10000

In [21]:
toks_ctr.most_common()

[('rt', 96229),
 ('via', 10616),
 ('new', 9388),
 ("n't", 7912),
 ('get', 5444),
 ('today', 5397),
 ('one', 5258),
 ('day', 4785),
 ('time', 4752),
 ('us', 4655),
 ('great', 4589),
 ('love', 4396),
 ('like', 4301),
 ('good', 3868),
 ('see', 3486),
 ('best', 3399),
 ('follow', 3389),
 ('gt', 3346),
 ('people', 3322),
 ('2', 3303),
 ('world', 3141),
 ('make', 3134),
 ('need', 3069),
 ('back', 3064),
 ('check', 3053),
 ('help', 2999),
 ('know', 2868),
 ('first', 2859),
 ('go', 2778),
 ("'m", 2766),
 ("'re", 2766),
 ('2016', 2762),
 ('thanks', 2700),
 ('free', 2691),
 ('want', 2574),
 ('1', 2507),
 ('last', 2494),
 ('year', 2489),
 ('live', 2406),
 ('3', 2405),
 ('win', 2393),
 ('work', 2386),
 ('4', 2378),
 ('week', 2346),
 ('happy', 2333),
 ('5', 2283),
 ('life', 2263),
 ('may', 2221),
 ('way', 2080),
 ('next', 2069),
 ('take', 2067),
 ('says', 2066),
 ('would', 2062),
 ('top', 2054),
 ('read', 2053),
 ('right', 2053),
 ('video', 2051),
 ('could', 2030),
 ('thank', 2011),
 ('big', 1981),

In [22]:
# Save out top 10000 words
with open('/usr2/mamille2/twitter/data/huang2016_data/train_text_vocab_10k.txt', 'w') as f:
    for w in vocab_10k:
        f.write(w + '\n')

In [23]:
# See number of words in tweets
tweet_lens = [len(d) for d in toks]
print(max(tweet_lens))
print(np.mean(tweet_lens))

82
19.0129045454


In [24]:
# Save out word embeddings for vocab
wembed10k = {w: wembed[w] for w in vocab_10k}

with open('/usr2/mamille2/twitter/data/huang2016_data/train_vocab_10k_emb.pkl', 'wb') as f:
    pickle.dump(wembed10k, f)

In [30]:
def preprocess_tweet(t):
    return [wd for wd in t if not wd in stops and not any([s in wd for s in stop_pats])]

In [86]:
# Save preprocessed tweet texts
toks = toks_train
toks_p = []

for d in tqdm(toks):
#     toks_p.append([t for t in d if t in vocab_all])
    d_p = preprocess_tweet(d)
    if len(d_p) > 0:
        toks_p.append(d_p)

print(len(toks_train))
print(len(toks_p))
print(toks_p[0])


215118
214999
['rt', 'walmsley_liz', 'morning', 'becky', 'might', 'surprised', 'know', 'going', 'walk', 'recce']


In [87]:
with open('/usr2/mamille2/twitter/data/huang2016_data/text_train.txt', 'w') as f:
    for d in toks_p:
        f.write(' '.join(d) + '\n')

In [84]:
%xdel wembed

In [30]:
MAX_LEN = 80

In [40]:
def tweet2vec(tweet_toks, max_len):
    """ Converts tags to indices, truncating and padding to length 5 (largest number of tags in the dataset is 4) """
    emb_list = [wembed10k[w] for w in tweet_toks[:max_len]]
    tweet_mat = np.pad(emb_list, ((0,max_len-len(emb_list)), (0,0)), 'constant', constant_values=0)
    
    return tweet_mat

In [44]:
eg = tweet2vec(['example', 'tweet'], MAX_LEN)
print(eg.shape)
print(eg)

(80, 300)
[[ 0.07404656 -0.16057926 -0.18220727 ..., -0.16359863 -0.10331138
   0.11113211]
 [-0.03699651 -0.20341067  0.00335062 ..., -0.04462308  0.49036312
  -0.69343126]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


In [45]:
# tweetmats_train = train_data['text_no_tags'].map(lambda x: tweet2vec(x, MAX_LEN)) # bad idea--too much memory

KeyboardInterrupt: 